In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/latest-data-science-job-salaries-2024/DataScience_salaries_2024.csv


In [28]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
import matplotlib.pyplot as plt
from lime.lime_tabular import LimeTabularExplainer
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor





In [29]:
df = pd.read_csv("/kaggle/input/latest-data-science-job-salaries-2024/DataScience_salaries_2024.csv")
df.shape[0]

14838

In [30]:
df.head(10)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2021,MI,FT,Data Scientist,30400000,CLP,40038,CL,100,CL,L
1,2021,MI,FT,BI Data Analyst,11000000,HUF,36259,HU,50,US,L
2,2020,MI,FT,Data Scientist,11000000,HUF,35735,HU,50,HU,L
3,2021,MI,FT,ML Engineer,8500000,JPY,77364,JP,50,JP,S
4,2022,SE,FT,Lead Machine Learning Engineer,7500000,INR,95386,IN,50,IN,L
5,2021,MI,FT,ML Engineer,7000000,JPY,63711,JP,50,JP,S
6,2021,SE,FT,Data Science Manager,7000000,INR,94665,IN,50,IN,L
7,2022,EN,FT,Data Scientist,6600000,HUF,17684,HU,100,HU,M
8,2022,EX,FT,Head of Machine Learning,6000000,INR,76309,IN,50,IN,L
9,2022,EN,FT,Research Engineer,5500000,JPY,41809,JP,50,JP,L


In [31]:
drop_columns = ["salary","salary_currency"]
df.drop(columns=drop_columns,inplace=True)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14838 entries, 0 to 14837
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           14838 non-null  int64 
 1   experience_level    14838 non-null  object
 2   employment_type     14838 non-null  object
 3   job_title           14838 non-null  object
 4   salary_in_usd       14838 non-null  int64 
 5   employee_residence  14838 non-null  object
 6   remote_ratio        14838 non-null  int64 
 7   company_location    14838 non-null  object
 8   company_size        14838 non-null  object
dtypes: int64(3), object(6)
memory usage: 1.0+ MB


In [33]:
# print(df.groupby('work_year').sum())
grouped_data = df.groupby('work_year').size().reset_index(name='count')

# Display the grouped data
print(grouped_data)

   work_year  count
0       2020     75
1       2021    218
2       2022   1652
3       2023   8519
4       2024   4374


**Encoding Data
**

In [34]:
df["work_year"].value_counts().head(15)

work_year
2023    8519
2024    4374
2022    1652
2021     218
2020      75
Name: count, dtype: int64

In [35]:
df["employment_type"].value_counts().head(10)

employment_type
FT    14772
PT       27
CT       26
FL       13
Name: count, dtype: int64

In [36]:
df = df[df["employment_type"]=="FT"].drop(columns=["employment_type"])


In [37]:
label_encoder = LabelEncoder()
df["experience_level"] = label_encoder.fit_transform(df["experience_level"])
df["company_size"] = label_encoder.fit_transform(df["company_size"])
df["job_title"] = label_encoder.fit_transform(df["job_title"])
df.head(10)

,work_year,experience_level,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2021,2,87,40038,CL,100,CL,0
1,2021,2,21,36259,HU,50,US,0
2,2020,2,87,35735,HU,50,HU,0
3,2021,2,112,77364,JP,50,JP,2
4,2022,3,111,95386,IN,50,IN,0
5,2021,2,112,63711,JP,50,JP,2
6,2021,3,84,94665,IN,50,IN,0
7,2022,0,87,17684,HU,100,HU,1
8,2022,1,106,76309,IN,50,IN,0
9,2022,0,142,41809,JP,50,JP,0


In [38]:
df["work_year"].value_counts().head(5)

work_year
2023    8503
2024    4354
2022    1638
2021     208
2020      69
Name: count, dtype: int64

In [39]:
residence_freq = df['employee_residence'].value_counts()
df['employee_residence_encoded'] = df['employee_residence'].map(residence_freq)

# Frequency encoding for company_location
company_freq = df['company_location'].value_counts()
df['company_location_encoded'] = df['company_location'].map(company_freq)

In [40]:
df.head(10)

,work_year,experience_level,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,employee_residence_encoded,company_location_encoded
0,2021,2,87,40038,CL,100,CL,0,2,1
1,2021,2,21,36259,HU,50,US,0,4,12938
2,2020,2,87,35735,HU,50,HU,0,4,3
3,2021,2,112,77364,JP,50,JP,2,8,8
4,2022,3,111,95386,IN,50,IN,0,72,57
5,2021,2,112,63711,JP,50,JP,2,8,8
6,2021,3,84,94665,IN,50,IN,0,72,57
7,2022,0,87,17684,HU,100,HU,1,4,3
8,2022,1,106,76309,IN,50,IN,0,72,57
9,2022,0,142,41809,JP,50,JP,0,8,8


In [41]:
df = pd.get_dummies(df,columns=["work_year"],prefix="year")

In [42]:
print(df["salary_in_usd"].values.reshape(-1,1)[:10])
print(df["salary_in_usd"].values.reshape(1,-1)[:10])

[[40038]
 [36259]
 [35735]
 [77364]
 [95386]
 [63711]
 [94665]
 [17684]
 [76309]
 [41809]]
[[40038 36259 35735 ... 15000 15000 15000]]


In [43]:
scaler = MinMaxScaler()
df["salary_in_usd"] = scaler.fit_transform(df["salary_in_usd"].values.reshape(-1,1))
                      #scaler.fit_transform(df["salary_in_usd"].values.reshape(-1, 1))


In [44]:
df.head(10)


,experience_level,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,employee_residence_encoded,company_location_encoded,year_2020,year_2021,year_2022,year_2023,year_2024
0,2,87,0.031896,CL,100,CL,0,2,1,False,True,False,False,False
1,2,21,0.027082,HU,50,US,0,4,12938,False,True,False,False,False
2,2,87,0.026414,HU,50,HU,0,4,3,True,False,False,False,False
3,2,112,0.079445,JP,50,JP,2,8,8,False,True,False,False,False
4,3,111,0.102403,IN,50,IN,0,72,57,False,False,True,False,False
5,2,112,0.062052,JP,50,JP,2,8,8,False,True,False,False,False
6,3,84,0.101484,IN,50,IN,0,72,57,False,True,False,False,False
7,0,87,0.003419,HU,100,HU,1,4,3,False,False,True,False,False
8,1,106,0.078101,IN,50,IN,0,72,57,False,False,True,False,False
9,0,142,0.034152,JP,50,JP,0,8,8,False,False,True,False,False


In [45]:
y = df["salary_in_usd"].values
X = df.drop(columns=["salary_in_usd","employee_residence","company_location"],axis=1)
print(type(y))

<class 'numpy.ndarray'>


In [46]:
print(y)

[0.03189554 0.02708153 0.02641401 ... 0.         0.         0.        ]


In [47]:
df.head(10)
# df = df[df["year_2024"]==True]
# df.drop(columns=["year_2020","year_2021","year_2022","year_2023"],inplace=True)
# df.head(10)



,experience_level,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,employee_residence_encoded,company_location_encoded,year_2020,year_2021,year_2022,year_2023,year_2024
0,2,87,0.031896,CL,100,CL,0,2,1,False,True,False,False,False
1,2,21,0.027082,HU,50,US,0,4,12938,False,True,False,False,False
2,2,87,0.026414,HU,50,HU,0,4,3,True,False,False,False,False
3,2,112,0.079445,JP,50,JP,2,8,8,False,True,False,False,False
4,3,111,0.102403,IN,50,IN,0,72,57,False,False,True,False,False
5,2,112,0.062052,JP,50,JP,2,8,8,False,True,False,False,False
6,3,84,0.101484,IN,50,IN,0,72,57,False,True,False,False,False
7,0,87,0.003419,HU,100,HU,1,4,3,False,False,True,False,False
8,1,106,0.078101,IN,50,IN,0,72,57,False,False,True,False,False
9,0,142,0.034152,JP,50,JP,0,8,8,False,False,True,False,False


In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
X_train.head(10)

,experience_level,job_title,remote_ratio,company_size,employee_residence_encoded,company_location_encoded,year_2020,year_2021,year_2022,year_2023,year_2024
6913,3,54,100,1,12899,12938,False,False,False,False,True
212,3,41,0,1,12899,12938,False,False,False,True,False
12992,2,54,100,1,12899,12938,False,False,False,False,True
1954,3,87,100,1,12899,12938,False,False,False,True,False
4281,3,87,0,1,12899,12938,False,False,False,True,False
654,3,43,0,1,12899,12938,False,False,False,True,False
2210,2,124,0,1,12899,12938,False,False,False,True,False
13805,0,60,0,1,12899,12938,False,False,False,False,True
2620,3,43,100,1,12899,12938,False,False,False,True,False
1400,3,54,0,1,12899,12938,False,False,False,True,False


In [51]:
y_train[:10]

array([0.16878981, 0.41759236, 0.07643312, 0.26547771, 0.21019108,
       0.33757962, 0.25808917, 0.05732484, 0.24394904, 0.28789809])

In [ ]:
models = {
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42)
}

In [ ]:
# Step 3: Loop through models to train, predict, and evaluate
results = {}

for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate loss metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    
    # Store results
    results[name] = {
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R² Score": r2
    }

# Step 4: Print results
for model_name, metrics in results.items():
    print(f"{model_name} Loss:")
    print(f"Mean Squared Error: {metrics['Mean Squared Error']:.2f}")
    print(f"Mean Absolute Error: {metrics['Mean Absolute Error']:.2f}\n")
    print(f"R² Score: {metrics['R² Score']:.2f}\n")

In [ ]:

# # Define the parameter grid
# rf = RandomForestRegressor()

# param_grid = {
#     'n_estimators': range(10, 300, 10),
#     'max_depth': [None, 10, 20, 30,40],
#     'min_samples_split': [2, 5, 10,20],
#     'min_samples_leaf': [1, 2, 4,8],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'criterion': ('friedman_mse', 'absolute_error'),

# }

# # Set up the grid search
# grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
#                            scoring='neg_mean_squared_error', 
#                            cv=3, n_jobs=-1, verbose=2)

# # Fit the model
# grid_search.fit(X_train, y_train)



In [ ]:
# grid_search.best_score_

In [ ]:
# grid_search.best_estimator_

In [ ]:
# Get the best parameters
# best_params = grid_search.best_params_
# print("Best Hyperparameters:", best_params)
print(X_train.count())
print(X_train.info())
print(X_train["experience_level"].isnull())

In [ ]:
model = RandomForestRegressor(
    criterion='friedman_mse', max_features='sqrt',
                      n_estimators=150,
    random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)


# Evaluate the model
# Assuming y_test and predictions are numpy arrays:
# Reshape if required
y_test = y_test.reshape(-1, 1)  # Reshape into column vector
predictions = predictions.reshape(-1, 1)

# Calculate metrics
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

# Print metrics
print("Mean Squared Error:", mse)
print("R² Score:", r2)
print("Mean Absolute Error:", mae)
print("Model Score:", model.score(X_test, y_test))


In [ ]:
# Single Instance prediction
predicted_Value = model.predict(X_test.head(1))
# print(y_test[0])
print(predicted_Value)
print("Y TEST : ",y_test[0])

In [ ]:
predicted_value = model.predict(X_test)
# print(y_test[0])
# print(predicted_Value.iloc[100])
print(type (predicted_value))
print(scaler.inverse_transform(predicted_value[-4].reshape(-1,1)))
print("Y TEST : ",scaler.inverse_transform(y_test[-4].reshape(-1,1)))

In [ ]:
# Access feature importances
importances = model.feature_importances_
print(importances)

feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
print(feature_importances)

In [ ]:
plt.figure(figsize=(10, 5))
plt.barh(feature_importances['Feature'], feature_importances['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Feature Importances from Random Forest Model')
plt.show()

# Features Explainibility using Lime


In [ ]:
explainer = LimeTabularExplainer(
     training_data=X_train.values,
     feature_names=X_train.columns,
     mode='regression'
)

# Explain the first instance in the test set
instance_to_explain = X_test.iloc[0].values.reshape(1, -1)
explanation = explainer.explain_instance(
    data_row=instance_to_explain[0],
    predict_fn=model.predict
)

In [ ]:
# Print the feature importances from LIME
print("Feature importances (LIME):")
for feature, importance in zip(X_train.columns, explanation.as_list()):
    print(f"{feature}: {importance}")

# Optionally, display the explanation in a notebook (if using Jupyter)
explanation.show_in_notebook(show_table=True, show_all=False)